# EVALUAR Y GENERAR REPORTS DE LAS ENCUESTAS


### CONSIDERACIONES PARA QUE FUNCIONE:

- HAY QUE INSTALAR LIBREOFFICE EN EL TERMINAL

        -  sudo apt update

        - sudo apt install libreoffice

- EL TEMPLATE DEL REPORT PARA EL PONENTE TIENE QUE ESTAR EN:

        - TEMPLATE_PATH = '/workspace/template/Evaluacion_Sesion_template.docx'

- FICHERO RESPUESTAS:


        - Eliminar las primeras 7 columnas del DataFrame, asumiendo que no son necesarias para el análisis.

        - Tiene que tener 8 preguntas numéricas de 0 a 5 y 4 con texto libre

        - Las preguntas 11 y 12 son las que hay que usar pare evaluar


In [7]:
%pip install docxtpl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from datetime import datetime
from docxtpl import DocxTemplate
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os
import subprocess
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import smtplib
import concurrent.futures
import os
from dotenv import load_dotenv

## PROCESA ENCUESTAS, GENERA REPORT Y ENVÍA EMAIL

In [9]:
def convert_to_pdf_with_libreoffice(docx_path, output_dir):
    """Convierte un archivo .docx a PDF usando LibreOffice en modo de línea de comandos"""
    try:
        # Ejecutar el comando libreoffice para convertir el archivo a PDF
        subprocess.run(['libreoffice', '--headless', '--convert-to', 'pdf', docx_path, '--outdir', output_dir], check=True)
        print(f"Archivo PDF generado a partir de: {docx_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error al convertir el archivo .docx a PDF: {e}")


In [10]:
def enviar_correo_con_adjunto(emails, nombre_sesion, pdf_path):
    """Envía un correo electrónico con el archivo PDF adjunto"""

    # Configurar el servidor SMTP (por ejemplo, Gmail)
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_user = os.getenv('SMTP_USER')  # Cargar desde .env
    smtp_password = os.getenv('SMTP_PASSWORD')  # Cargar desde .env

    # Crear el mensaje
    asunto = f"Resultados encuestas sesión {nombre_sesion}"
    mensaje = f"""
    Hola,

    Quería agradecerte de corazón por participar en la sesión "{nombre_sesion}". Tu charla fue de gran valor para los alumnos, quienes realmente apreciaron todo lo que compartiste. No solo compartiste información valiosa, sino que también les ayudaste a entender mejor el tema desde una perspectiva práctica y clara.

    Te adjunto los resultados de la encuesta de satisfacción para que veas lo mucho que valoraron tu participación. Estoy seguro de que tus aportes marcaron una diferencia para ellos.

    Espero que te haya resultado una buena experiencia y que podamos contar contigo en futuras ocasiones.

    Un abrazo,
    
    Alfonso.
    
    """

    # Crear el objeto MIME para el mensaje
    msg = MIMEMultipart()
    msg['From'] = smtp_user
    msg['To'] = ', '.join(emails) if isinstance(emails, list) else emails
    msg['Subject'] = asunto

    # Adjuntar el texto del mensaje
    msg.attach(MIMEText(mensaje, 'plain'))

    # Adjuntar el archivo PDF
    with open(pdf_path, 'rb') as f:
        part = MIMEApplication(f.read(), _subtype='pdf')
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(pdf_path))
        msg.attach(part)

    # Enviar el correo
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Seguridad TLS
            server.login(smtp_user, smtp_password)
            server.sendmail(smtp_user, emails if isinstance(emails, list) else [emails], msg.as_string())
            print(f"Correo enviado con éxito a {emails}")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

In [11]:
# Constante para la ruta de la plantilla
TEMPLATE_PATH = '/workspace/template/Evaluacion_Sesion_template.docx'

def procesar_encuesta(file_path_csv, configuracion, output_path):
    try:
        # Verificar si el archivo CSV existe, si no, lanzar un error.
        if not os.path.exists(file_path_csv):
            raise FileNotFoundError(f"El archivo CSV {file_path_csv} no existe.")
        
        # Leer los datos del CSV en un DataFrame de pandas.
        df = pd.read_csv(file_path_csv)

        # Verificar si el DataFrame está vacío. Si está vacío, lanzar un error.
        if df.empty:
            raise ValueError("El archivo CSV está vacío o no contiene datos válidos.")

        # Eliminar las primeras 7 columnas del DataFrame, asumiendo que no son necesarias para el análisis.
        columns_to_drop = df.columns[:7]
        df = df.drop(columns=columns_to_drop)

        # Obtener la fecha actual en formato 'día/mes/año'.
        fecha_hoy = datetime.today().strftime('%d/%m/%Y')

        # Crear un diccionario llamado 'context' para almacenar los datos que se usarán para rellenar la plantilla.
        context = {
            "Titulo_sesion": configuracion['nombre_sesion'],  # Asigna el nombre de la sesión.
            "fecha": fecha_hoy,  # Asigna la fecha actual.
        }

        # Definir las columnas del CSV que contienen respuestas numéricas y asignarles un sufijo de marcador de posición.
        columnas = [
            ("Q01_Recomendar->1- Nunca, 5- Siempre", "t1"),
            ("Q02_Evaluación General->1-Nada, 5- Mucho", "t2"),
            ("Q03_General Presentador->1- Muy malo, 5- Muy bueno", "t3"),
            ("Q04_Se preocupa->1-Nada, 5-Mucho", "t4"),
            ("Q05_Nivel presentador->1- Muy poco, 5- Mucho", "t5"),
            ("Q06_Tema interesante->1- Muy poco, 5-Mucho", "t6"),
            ("Q07_Manera estructurada->1- Muy poco, 5- Mucho", "t7"),
            ("Q08_Respuesta preguntas->1- Muy mal, 5-Muy bien", "t8")
        ]

        # Para cada columna de respuestas, contar cuántas veces aparece cada valor y almacenarlo en el diccionario 'context'.
        for columna, sufijo in columnas:
            conteos = df[columna].value_counts().to_dict()  # Contar las ocurrencias de cada valor en la columna.
            for valor in range(6):  # Asume que las respuestas varían de 0 a 5.
                placeholder = f"respuestas_{valor}_{sufijo}"  # Crear un marcador de posición en el formato 'respuestas_X_tY'.
                context[placeholder] = conteos.get(valor, 0)  # Asignar el conteo de respuestas o 0 si no existen.

        # Crear listas de respuestas para las preguntas abiertas, usando viñetas en lugar de saltos de línea simples.
        context['Q09_Gustado'] = '\n'.join([f"• {item}\n" for item in df['Q09_Gustado'].dropna().tolist()])
        context['Q10_Cambios'] = '\n'.join([f"• {item}\n" for item in df['Q10_Cambios'].dropna().tolist()])
        context['Pregunta_1'] = configuracion['pregunta_1']  # Asignar el valor de la primera pregunta.
        context['Q11_Especifica1'] = '\n'.join([f"• {item}\n" for item in df['Q11_Específica 1'].dropna().tolist()])
        context['Pregunta_2'] = configuracion['pregunta_2']  # Asignar el valor de la segunda pregunta.
        context['Q12_Especifica2'] = '\n'.join([f"• {item}\n" for item in df['Q12_Específica 2'].dropna().tolist()])

        # Verificar si el archivo de plantilla de Word existe, si no, lanzar un error.
        if not os.path.exists(TEMPLATE_PATH):
            raise FileNotFoundError(f"El archivo de plantilla {TEMPLATE_PATH} no existe.")

        # Cargar la plantilla de Word utilizando DocxTemplate.
        doc = DocxTemplate(TEMPLATE_PATH)

        # Rellenar la plantilla de Word con los datos almacenados en el diccionario 'context'.
        doc.render(context)

        # Centrar el texto dentro de las celdas de todas las tablas del documento.
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Centrar el texto en las celdas.

        # Guardar el documento generado en formato .docx en la ruta especificada por 'output_path'.
        doc.save(output_path)
        
        # Generar la ruta del archivo PDF reemplazando la extensión .docx por .pdf.
        output_pdf_path = output_path.replace('.docx', '.pdf')
        output_dir = os.path.dirname(output_pdf_path)  # Obtener el directorio de salida del PDF.

        # Convertir el archivo .docx a PDF utilizando LibreOffice.
        convert_to_pdf_with_libreoffice(output_path, output_dir)

        # Imprimir un mensaje de finalización exitosa y mostrar la ruta del archivo PDF.
        print("Proceso finalizado con éxito. El documento PDF ha sido guardado en:", output_pdf_path)

        # Enviar el archivo PDF generado por correo electrónico como adjunto.
        enviar_correo_con_adjunto(configuracion['email'], configuracion['nombre_sesion'], output_pdf_path)

    # Capturar el error si el archivo CSV o la plantilla no existen.
    except FileNotFoundError as fnf_error:
        print(fnf_error)

    # Capturar el error si el archivo CSV está vacío o no contiene datos válidos.
    except ValueError as val_error:
        print(val_error)

    # Capturar cualquier otro error inesperado y mostrar un mensaje de error.
    except Exception as e:
        print("Ocurrió un error inesperado:", str(e))


In [12]:
configuracion = {
    'nombre_sesion': 'IA Aplicada a las Finanzas',
    'pregunta_1': '¿Cuáles son las cuatro áreas de conocimiento de la IA?',
    'pregunta_2': '¿Qué es un algoritmo de mejor ejecución?',
    'email': 'alfonso.santos@uam.es'
}

file_path_csv = "/workspace/data/EVALUACI_N_SESI_N_GUILLERMO_2311.csv"
output_path = f'/workspace/reports/{configuracion["nombre_sesion"]}/Informe_{configuracion["nombre_sesion"]}.docx'


procesar_encuesta(file_path_csv, configuracion, output_path)


convert /workspace/reports/IA Aplicada a las Finanzas/Informe_IA Aplicada a las Finanzas.docx -> /workspace/reports/IA Aplicada a las Finanzas/Informe_IA Aplicada a las Finanzas.pdf using filter : writer_pdf_Export
Archivo PDF generado a partir de: /workspace/reports/IA Aplicada a las Finanzas/Informe_IA Aplicada a las Finanzas.docx
Proceso finalizado con éxito. El documento PDF ha sido guardado en: /workspace/reports/IA Aplicada a las Finanzas/Informe_IA Aplicada a las Finanzas.pdf
Correo enviado con éxito a alfonso.santos@uam.es
